# Rating and Reviews Data Wrangling

**PACKAGES**

In [1]:
import os
import pickle
import pandas as pd
import numpy as np
import time
from fuzzywuzzy import fuzz
import textdistance

C:\Users\asaid\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


**CODE PARAMETERS**

In [2]:
# PANDAS DISPLAY PARAMETERS
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', 1)

In [3]:
# PATHS FOR RATINGS AND REVIEWS
output_ratings_path = r'../data/clean_data/'
input_rating_paths = os.path.join(os.getcwd(),r'../data/raw_data/Rating and Reviews/')

# PATHS FOR PRODUCT CATALOGUE
output_product_path = r'../data/clean_data/'
input_product_paths = os.path.join(os.getcwd(),r'../data/raw_data/Product Catalogue/')

# PATHS FOR DEMAND
output_demand_path = r'../data/clean_data/'
input_demand_paths = os.path.join(os.getcwd(),r'../data/raw_data/Demand Data/')

## Aggregating Product Catalogue Data

### Reading and aggregating data

We have a look at the available data

In [4]:
os.listdir(input_product_paths)

['Cosmetics_Product_20190831.csv',
 'Cosmetics_Product_20190930.csv',
 'Cosmetics_Product_20191031.csv',
 'cosmetics_product_2019Q4.csv',
 'Cosmetics_Product_20200116.csv',
 'Global Product Catalogs - ELC ST Fix',
 'Historic Product Catalog - ELC ST Fix.b',
 'Historic Product Catalogs - ELC ST Fix',
 'SkinCare_New_Product_20191031.zip',
 'SkinCare_Product_20190630_ST MV2SV.csv',
 'SkinCare_Product_20190930.csv',
 'SkinCare_Product_20190930_Update.csv',
 'SkinCare_Product_20191031.csv',
 'skincare_product_20191231.csv',
 'skincare_product_2019Q3.csv',
 'skincare_product_2019Q3_final.csv',
 'SkinCare_Product_20200116.csv']

Since I don't want to create mappings between product catalogue files and rating and reviews files I will just concatenated the product catalogue files in one big catalogue and use it to left merge on with the rating and reviews data.

We create a DataFrame for skincare and cosmetics that concatenates all the dataframes.

In [5]:
product_skincare = pd.DataFrame()
product_cosmetics = pd.DataFrame()
for file in os.listdir(input_product_paths):
    if '.csv' in file.lower():
        if 'skincare' in file.lower():
            print('Adding', file)
            temp = pd.read_csv(os.path.join(input_product_paths,file))
            temp = temp.loc[:, ~temp.columns.str.contains('^Unnamed')]
            product_skincare=pd.concat([product_skincare, temp])
        elif 'cosmetics' in file.lower():
            print('Adding', file)
            temp = pd.read_csv(os.path.join(input_product_paths,file))
            temp = temp.loc[:, ~temp.columns.str.contains('^Unnamed')]
            product_cosmetics=pd.concat([product_cosmetics, temp])
del temp

Adding Cosmetics_Product_20190831.csv
Adding Cosmetics_Product_20190930.csv
Adding Cosmetics_Product_20191031.csv
Adding cosmetics_product_2019Q4.csv
Adding Cosmetics_Product_20200116.csv
Adding SkinCare_Product_20190630_ST MV2SV.csv
Adding SkinCare_Product_20190930.csv


C:\Users\asaid\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (12,13,16,17,18,26,29,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,163,164,168,169,170,171,172) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Adding SkinCare_Product_20190930_Update.csv


C:\Users\asaid\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Adding SkinCare_Product_20191031.csv
Adding skincare_product_20191231.csv
Adding skincare_product_2019Q3.csv
Adding skincare_product_2019Q3_final.csv
Adding SkinCare_Product_20200116.csv


### Formating Skincare Prodcut Catalogue data

We start by changing the columns names to make them code friendly

In [ ]:
product_skincare.columns = [colname.lower().replace(' ','_') for colname in product_skincare.columns]

We format the Clean Collection Date and fill the missing rows/bad format with the oldest date available.

In [2]:
product_skincare['clean_collection_date'] = pd.to_datetime(product_skincare['collection_date'], errors='coerce')
product_skincare.loc[product_skincare['clean_collection_date'].isna(), 'collection_date']=min(product_skincare['clean_collection_date'])
product_skincare['clean_collection_date'] = pd.to_datetime(product_skincare['collection_date'], errors='coerce')

NameError: name 'pd' is not defined

We drop the duplicates while only keeping the row with the most recent Collection Date

In [8]:
product_skincare = product_skincare.sort_values('clean_collection_date', ascending=False).dropna(subset=['brand']).drop_duplicates([
    'elc_solution_type',
    'source_product_identifier', 
    'product_id'])

Finally, we save the DataFrame:

In [9]:
product_skincare.to_csv(os.path.join(output_product_path,'product_catalogue_skincare.csv'), index=False)

### Formating Cosmetics Prodcut Catalogue data

We do the same for Coemetics. We start by changing the columns names to make them code friendly

In [10]:
product_cosmetics.columns = [colname.lower().replace(' ','_') for colname in product_cosmetics.columns]

We format the Clean Collection Date and fill the missing rows/bad format with the oldest date available.

In [11]:
product_cosmetics['clean_collection_date'] = pd.to_datetime(product_cosmetics['collection_date'], errors='coerce')
product_cosmetics.loc[product_cosmetics['clean_collection_date'].isna(), 'collection_date']=min(product_cosmetics['clean_collection_date'])
product_cosmetics['clean_collection_date'] = pd.to_datetime(product_cosmetics['collection_date'], errors='coerce')

We drop the duplicates while only keeping the row with the most recent Collection Date

In [12]:
product_cosmetics.sort_values('clean_collection_date', ascending=False).dropna(subset=['brand']).drop_duplicates([
    'elc_solution_type',
    'source_product_identifier', 
    'product_id'], inplace=True)

Finally, we save the DataFrame:

In [13]:
product_cosmetics.to_csv(os.path.join(output_product_path,'product_catalogue_cosmetics.csv'), index=False)

## Aggregating Ratings and Reviews Data

Here are some ideas about the aggregationg of R&R data :
1. Filter only on US
2. create data by YM
3. change sentiment to num_sentiment : -1,0,1
4. change sentiments to hot ones HO_sentiment
5. create hot ones rating
5. Groupby YM, Source Product Identifierm, Channel, Product_ID
6. Aggregate by {num_sentiment:mean, num_rating:mean, HO_sentiments:count, HO_rating:count}
7. Number of reviews

In [16]:
def format_rating_and_reviews(rating_reviews, product_catalogue):
    """
    Formats the raw rating and reviews dataset and adds the brand using the product_catalogue dataset.
    :param rating_reviews: DataFrame: Rating and reviews dataframe.
    :param product_catalogue: DataFrame: Product catalogue dataframe.
    :return: DataFrame: Aggregated RR data with the column 'brand'.
    """
    print('Formatting data...')
    # Creating column friendly columns
    rating_reviews.columns = [colname.lower().replace(' ','_') for colname in rating_reviews.columns]
    # Filtering geographies on US
    rating_reviews = rating_reviews[rating_reviews['geography']=='USA']
    # Creating date columns in the right dtype.
    rating_reviews.loc[:,'clean_date'] = pd.to_datetime(rating_reviews['date'], errors='coerce')
    rating_reviews.loc[:,'year'] = rating_reviews.loc[:,'clean_date'].dt.year
    rating_reviews.loc[:,'month'] = rating_reviews.loc[:,'clean_date'].dt.month
    # Formating the rating and sentiment (NA => -1)
    rating_reviews.loc[:,'rating'] = rating_reviews['rating'].fillna(-1).astype(int)
    rating_reviews.loc[:,'sentiment'] = rating_reviews['sentiment'].str.lower()
    # Transforming rating and sentiment to dummy variables (one-hot encoding)
    rating_reviews = pd.concat([rating_reviews, pd.get_dummies(data=rating_reviews[['rating','sentiment']], columns=['rating','sentiment'], dtype=int)], axis=1)
    # THE FOLLOWING IS OPTIONAL :
    # Transforming sentiment to integer data (positive:1; netural:0, negative:-1)
    rating_reviews.loc[:,'sentiment'] = rating_reviews['sentiment_positive'] - rating_reviews['sentiment_negative']
    # Creating a column to count the number of reviews once aggreagtion happens
    rating_reviews['nb_reviews'] = rating_reviews['rating']
    # Readding NA data to ratings
    rating_reviews.loc[rating_reviews['rating']==-1,'rating'] = np.nan
    
    
    # Aggregating RR data
    print('Aggregating data...')
    # TODO: Try adding 'channel' to groupby !
    rating_reviews = rating_reviews.groupby(['elc_solution_type',
                     'source_product_identifier', 
                     'product_id', 
                     'year', 
                     'month']).agg({
        'nb_reviews':'count',
        'rating':'mean',
        'rating_1':'sum',
        'rating_2':'sum',
        'rating_3':'sum',
        'rating_4':'sum',
        'rating_5':'sum',
        'sentiment_negative':'sum',
        'sentiment_neutral':'sum',
        'sentiment_positive':'sum',
        'sentiment':'mean'
    }).reset_index()
    
    
    print('Adding product catalogue data...')
    # Formating Product Catalogue data
    initial_size=len(rating_reviews)
    product_catalogue = product_catalogue[['elc_solution_type',
                                           'source_product_identifier', 
                                           'product_id',
                                           'brand']].drop_duplicates(['elc_solution_type',
                                                                      'source_product_identifier', 
                                                                      'product_id'])
    
    # Merging PC and RR to add brand (we use different level of aggregation to avoid having NAs)
    rating_reviews = rating_reviews.merge(product_catalogue[['elc_solution_type','source_product_identifier', 'product_id', 'brand']], how='left')
    rating_reviews['brand'].fillna(rating_reviews.merge(product_catalogue[['elc_solution_type','source_product_identifier','brand']].drop_duplicates(), on=['elc_solution_type','source_product_identifier'], how='left', suffixes=['','_filler'])['brand_filler'], inplace=True)
    rating_reviews['brand'].fillna(rating_reviews.merge(product_catalogue[['source_product_identifier','brand']].drop_duplicates(), on=['source_product_identifier'], how='left', suffixes=['','_filler'])['brand_filler'], inplace=True)
    
    # Checks that the merge generated no duplicates and prints missing data % (if any)
    print('Check that no duplicates have been created:', initial_size==len(rating_reviews))
    if rating_reviews.isna().sum().sum()>0:
        print('Missing values:')
        display(rating_reviews.isna().sum()/len(rating_reviews)*100)
        
    return rating_reviews

Let's have a look at the datafiles we have :

In [18]:
os.listdir(input_rating_paths)

['Cosmetics_Reviews_20190630.csv',
 'Cosmetics_Reviews_20190831.csv',
 'Cosmetics_Reviews_20190930.csv',
 'Cosmetics_Reviews_20191031.csv',
 'cosmetics_reviews_20191130_final.csv',
 'cosmetics_reviews_20191231.csv',
 'cosmetics_reviews_20200101-20200131.csv',
 'cosmetics_reviews_20200229.csv',
 'deleted',
 'pwds',
 'Skin Care_Reviews_20190930.csv',
 'skincare_reviews_20150201-20200131.csv',
 'SkinCare_Reviews_20190630_ST MV2SV.csv',
 'SkinCare_Reviews_20190831.csv',
 'SkinCare_Reviews_20191031.csv',
 'skincare_reviews_20191130.csv',
 'skincare_reviews_20191231.csv',
 'skincare_reviews_20200229.csv']

In [22]:
# USE THIS IF READING DATA IS TOO SLOW OR IF YOU RUN INTO THE FOLLOWING WARNING :
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.errors.DtypeWarning.html

types = {'OnlinePost_ID': object, 
    'Source Product Identifier': object,
    'OnlineStatement_ID': object,
    'Date': object,
    'Title': object,
    'Description': object,
    'Geography': object,
    'Channel': object,
    'Product_ID': object,
    'Rating': float,
    'Sentiment': object,
    'Feature': object,
    'Benefit': object,
    'Ingredient': object,
    'Additional Ingredients (no rulebase)': object,
    'Product Form': object,
    'ELC Solution Type': object,
    'Finish': object,
    'Looks': object,
    'Other': object,
    'Trends': object,
    'Syndication Source': object,
    'Best For': object,
    'Verified Buyer': object,
    'From': object,
    'Recommended': object,
    'Verified Reviewer': object,
    'Eye Color': object,
    'Hair Color': object,
    'Skin Tone': object,
    'Gender': object,
    'I shop at macys.com': object,
    'Make-up Style': object,
    'Purchase Location': object,
    'Cons': object,
    'Pros': object,
    'Describe Yourself': object,
    'Reviewer Skin Type': object,
    'Age': object}

Then we calculate then number of files in each section (I use them to print the progession because I hate waiting without knowing how much is left to do !)

In [19]:
nb_files_to_read_skincare = 0
nb_files_to_read_cosmetics = 0
for file in os.listdir(input_rating_paths):
    if '.csv' in file.lower() :
        if 'skincare' in file.lower():
            nb_files_to_read_skincare+=1
        elif 'cosmetics' in file.lower():
            nb_files_to_read_cosmetics+=1
print('Number of skincare files to read:', nb_files_to_read_skincare)
print('Number of cosmetics files to read:', nb_files_to_read_cosmetics)

Number of skincare files to read: 7
Number of cosmetics files to read: 8


### Concatenating Skincare data

In [23]:
# Use this incase you didn't run section 1
#product_skincare = pd.read_csv(os.path.join(output_product_path,'product_catalogue_skincare.csv'))

In [21]:
start = time.time()
nb_read_skincare = 0
ratings_skincare = pd.DataFrame()
   
for file in os.listdir(input_rating_paths):
    if '.csv' in file.lower() and 'skincare' in file.lower():
        interm = time.time()
        nb_read_skincare+=1
        print('Reading', file, nb_read_skincare,'out of', nb_files_to_read_skincare, '...')
        temp = pd.read_csv(os.path.join(input_rating_paths,file))
        temp = temp.loc[:, ~temp.columns.str.contains('^Unnamed')]
        temp = format_rating_and_reviews(temp, product_skincare)
        ratings_skincare = pd.concat([ratings_skincare, temp])
        print('New length skincare:', len(ratings_skincare))
        print('Time for this dataset:', time.time()-interm)
        print('Total time:', time.time()-start)
del temp
ratings_skincare.to_csv(os.path.join(output_ratings_path,'ratings_and_reviews_skincare_by_product_id.csv'), index=False)

Reading skincare_reviews_20150201-20200131.csv 1 out of 7 ...


C:\Users\asaid\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (23,25,28,29,30,33,34,35,36,37,38) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Formatting data...


C:\Users\asaid\Anaconda3\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\asaid\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Aggregating data...
Adding product catalogue data...
Check that no duplicates have been created: True
New length skincare: 291982
Time for this dataset: 47.46156287193298
Total time: 47.46255564689636
Reading SkinCare_Reviews_20190630_ST MV2SV.csv 2 out of 7 ...


...

Formatting data...
Aggregating data...
Adding product catalogue data...
Check that no duplicates have been created: True
New length skincare: 887177
Time for this dataset: 355.12283515930176
Total time: 478.4536974430084
Reading skincare_reviews_20191130.csv 5 out of 7 ...


C:\Users\asaid\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (1,23,24,25,26,27,28,29,30,32,33,34,35,36,37,38,39) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Formatting data...
Aggregating data...
Adding product catalogue data...
Check that no duplicates have been created: True
New length skincare: 1201707
Time for this dataset: 193.73127436637878
Total time: 672.185962677002
Reading skincare_reviews_20191231.csv 6 out of 7 ...


C:\Users\asaid\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (33,34) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Formatting data...
Aggregating data...
Adding product catalogue data...
Check that no duplicates have been created: True
New length skincare: 1210068
Time for this dataset: 4.572640419006348
Total time: 676.7595958709717
Reading skincare_reviews_20200229.csv 7 out of 7 ...
Formatting data...
Aggregating data...
Adding product catalogue data...
Check that no duplicates have been created: True
New length skincare: 1220339
Time for this dataset: 11.561578512191772
Total time: 688.3211743831635


### Concatenating Cosmetics data

In [24]:
product_cosmetics = pd.read_csv(os.path.join(output_product_path,'product_catalogue_cosmetics.csv'))

C:\Users\asaid\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (21,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [25]:
start = time.time()
nb_read_cosmetics = 0
ratings_cosmetics = pd.DataFrame()
for file in os.listdir(input_rating_paths):
    if '.csv' in file.lower() and 'cosmetics' in file.lower():
        interm = time.time()
        nb_read_cosmetics+=1
        print('Reading', file, nb_read_cosmetics,'out of', nb_files_to_read_cosmetics, '...')
        temp = pd.read_csv(os.path.join(input_rating_paths,file))
        temp = temp.loc[:, ~temp.columns.str.contains('^Unnamed')]
        temp = format_rating_and_reviews(temp, product_cosmetics)
        ratings_cosmetics = pd.concat([ratings_cosmetics, temp])
        print('New length cosmetic:', len(ratings_cosmetics))
        print('Time for this dataset:', time.time()-interm)
        print('Total time:', time.time()-start)
del temp
ratings_cosmetics.to_csv(os.path.join(output_ratings_path,'ratings_and_reviews_cosmetics_by_product_id.csv'), index=False)

Reading Cosmetics_Reviews_20190630.csv 1 out of 8 ...
Formatting data...
Aggregating data...
Adding product catalogue data...
Check that no duplicates have been created: True
Missing values:


elc_solution_type            0.000000
source_product_identifier    0.000000
product_id                   0.000000
year                         0.000000
month                        0.000000
rating                       0.000000
rating_1                     0.000000
rating_2                     0.000000
rating_3                     0.000000
rating_4                     0.000000
rating_5                     0.000000
sentiment_negative           0.000000
sentiment_neutral            0.000000
sentiment_positive           0.000000
sentiment                    0.000000
brand                        0.604901
dtype: float64

New length cosmetic: 48603
Time for this dataset: 51.22840762138367
Total time: 51.22986149787903
Reading Cosmetics_Reviews_20190831.csv 2 out of 8 ...
Formatting data...
Aggregating data...
Adding product catalogue data...
Check that no duplicates have been created: True
New length cosmetic: 66647
Time for this dataset: 4.765073776245117
Total time: 55.994935274124146
Reading Cosmetics_Reviews_20190930.csv 3 out of 8 ...


C:\Users\asaid\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (16,22) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Formatting data...
Aggregating data...
Adding product catalogue data...
Check that no duplicates have been created: True
New length cosmetic: 291592
Time for this dataset: 123.15586876869202
Total time: 179.15129971504211
Reading Cosmetics_Reviews_20191031.csv 4 out of 8 ...
Formatting data...
Aggregating data...
Adding product catalogue data...
Check that no duplicates have been created: True
New length cosmetic: 643380
Time for this dataset: 261.9915351867676
Total time: 441.1438031196594
Reading cosmetics_reviews_20191130_final.csv 5 out of 8 ...


...

Formatting data...
Aggregating data...
Adding product catalogue data...
Check that no duplicates have been created: True
New length cosmetic: 1011049
Time for this dataset: 21.213435888290405
Total time: 1109.2639582157135
Reading cosmetics_reviews_20200101-20200131.csv 7 out of 8 ...


C:\Users\asaid\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (27,28,29,32,33,37) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Formatting data...
Aggregating data...
Adding product catalogue data...
Check that no duplicates have been created: True
New length cosmetic: 1021825
Time for this dataset: 5.178733825683594
Total time: 1114.442692041397
Reading cosmetics_reviews_20200229.csv 8 out of 8 ...


C:\Users\asaid\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (22,27,28,29,32,33,34,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Formatting data...
Aggregating data...
Adding product catalogue data...
Check that no duplicates have been created: True
New length cosmetic: 1031379
Time for this dataset: 5.139551639556885
Total time: 1119.582738161087


### Merging  Skincare and Cosmetics

In [79]:
# Concatenating RR reviews
ratings_skincare['major_category'] = 'Skincare'
ratings_cosmetics['major_category'] = 'Cosmetics'
ratings = pd.concat([ratings_skincare, ratings_cosmetics])
# We set brands to all lower case to avoid duplicated (Bobby Brown/obby brown)
ratings['brand'] = ratings['brand'].str.lower()

In [80]:
ratings

,elc_solution_type,source_product_identifier,product_id,year,month,rating,rating_1,rating_2,rating_3,rating_4,rating_5,sentiment_negative,sentiment_neutral,sentiment_positive,sentiment,brand,major_category
0,After Sun,1980608,Product_20200109_15497007,2020,1,5.000000,0,0,0,0,4,0,0,4,1.000000,coola,Skincare
1,After Sun,3792419,Product_20200112_15508995,2020,1,5.000000,0,0,0,0,4,0,0,4,1.000000,coola,Skincare
2,After Sun,5172,Product_20200109_14157574,2020,1,5.000000,0,0,0,0,3,0,0,3,1.000000,coola,Skincare
3,After Sun,B000052YFY,Product_20191016_13660574,2015,2,4.000000,0,0,0,2,0,0,0,2,1.000000,banana boat,Skincare
4,After Sun,B000052YFY,Product_20191016_13660574,2017,8,2.000000,0,1,0,0,0,0,0,1,1.000000,banana boat,Skincare
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9549,Unspecified Lip Makeup,P444926,Product_20200109_5826612,2020,2,3.631579,0,6,0,8,5,3,2,14,0.578947,fenty beauty,Cosmetics
9550,Unspecified Lip Makeup,P449848,Product_20200109_5826598,2020,2,4.333333,0,0,0,4,2,0,0,6,1.000000,innisfree,Cosmetics
9551,Unspecified Lip Makeup,P450585,Product_20200109_5826604,2020,2,5.000000,0,0,0,0,3,0,0,3,1.000000,kopari beauty,Cosmetics
9552,Unspecified Lip Makeup,pimprod2005621,Product_20200109_5852942,2020,2,5.000000,0,0,0,0,21,1,0,20,0.904762,weleda,Cosmetics


In [81]:
ratings.to_csv(os.path.join(output_ratings_path,'ratings_and_reviews_by_product_id.csv'), index=False)

## Mapping Brands

The brands displayed in the Product Catalogue data are not the official Estée Lauder format. We have a mapping of Estée Lauder brands and brands abbreviation, which we will map to the Product Catalogue brands.

In [97]:
# Estée Lauder brands
# A better way to get these, is from the Demand data (get brand_abbrev then manually add elc_brand).
# OR ask Anurag to update the full list (I feel like some brands are missing...)
elc_brands = pd.read_csv('../data/clean_data/elc_brands.csv', encoding = "ISO-8859-1")
elc_brands

,brand_abbrev,elc_brand
0,CL,Clinique
1,MC,M.A.C
2,EL,Estée Lauder
3,AV,Aveda
4,SX,Smashbox
5,OR,Origins
6,BB,Bobbi Brown
7,AR,Aramis
8,JM,Jo Malone London
9,BU,Bumble and bumble


In [29]:
# Brands from Product Catalogue
brands = pd.DataFrame({'brand' : ratings['brand'].unique()})
brands

,brand
0,coola
1,banana boat
2,weleda
3,alba botanica
4,maui babe
...,...
1583,thrive cosmetics
1584,artistry youth xtend
1585,guerlain meteorites base
1586,gibson


In [30]:
# We create all possible combinations between PC Brands and EL Brands.
brand_matching = brands.assign(key=0).merge(elc_brands.assign(key=0), on='key', how='left').drop('key', axis=1)
brand_matching

,brand,brand_abbrev,elc_brand
0,coola,CL,Clinique
1,coola,MC,M.A.C
2,coola,EL,Estée Lauder
3,coola,AV,Aveda
4,coola,SX,Smashbox
...,...,...,...
33343,schwarzkopf,PR,Prescriptives
33344,schwarzkopf,FM,Frédéric Malle
33345,schwarzkopf,IM,Non-brand
33346,schwarzkopf,BA,BECCA


In [31]:
def custom_distance(row):
    """
    Calculates a distance score between two sentences. In this case elc_brand and brand. The score is between 0 and 1, 1 being a good match.
    """
    jaro = textdistance.jaro_winkler(str(row['brand']).lower().replace('.','').replace('&','and'), str(row['elc_brand']).lower().replace('.',''))
#     jaccard = textdistance.jaccard(str(row['brand']).lower().replace('.','').replace('&','and'), str(row['elc_brand']).lower().replace('.',''))
    fuzzi = fuzz.partial_ratio(str(row['brand']).lower().replace('.','').replace('&','and'), str(row['elc_brand']).lower().replace('.',''))/100
    return (fuzzi+jaro)/2

In [32]:
# Calculate the matching score between brand and elc_brand
brand_matching['score'] = brand_matching.apply(lambda row : custom_distance(row), axis=1)
brand_matching

,brand,brand_abbrev,elc_brand,score
0,coola,CL,Clinique,0.475000
1,coola,MC,M.A.C,0.200000
2,coola,EL,Estée Lauder,0.461111
3,coola,AV,Aveda,0.333333
4,coola,SX,Smashbox,0.345833
...,...,...,...,...
33343,schwarzkopf,PR,Prescriptives,0.426888
33344,schwarzkopf,FM,Frédéric Malle,0.297280
33345,schwarzkopf,IM,Non-brand,0.225673
33346,schwarzkopf,BA,BECCA,0.463636


In [33]:
brand_matching.to_csv('../data/clean_data/brand_mapping_scores.csv', index=False)

In [34]:
# Keep the elc_brand that has the highest score
brand_matching = brand_matching.groupby('brand').apply(lambda x: x.nlargest(1,'score')).reset_index(drop=True)
brand_matching

,brand,brand_abbrev,elc_brand,score
0,/skin regimen/,-,,0.845238
1,100% pure,-,,0.500000
2,1001 remedies,-,,0.846154
3,111skin,KL,By Kilian,0.508651
4,2(x)ist,AR,Aramis,0.434841
...,...,...,...,...
1582,zhangwei,GG,GlamGlow,0.506667
1583,zia botanicals,-,,0.845238
1584,ziip beauty,-,,0.848485
1585,zitsticka,KL,By Kilian,0.477778


In [35]:
# Select a threshhold
brand_matching.loc[brand_matching['score']>0.92]

,brand,brand_abbrev,elc_brand,score
89,aramis,AR,Aramis,1.000000
117,aveda,AV,Aveda,1.000000
175,becca,BA,BECCA,1.000000
176,becca cosmetics,BA,BECCA,0.933333
230,bobbi brown,BB,Bobbi Brown,1.000000
255,bumble & bumble,BU,Bumble and bumble,1.000000
256,bumble and bumble,BU,Bumble and bumble,1.000000
319,clinique,CL,Clinique,1.000000
320,clinique;cliniquee,CL,Clinique,0.944444
356,darphin,DA,Darphin,1.000000


In [36]:
# All matching that have a score below the threshhold will be unmatched
brand_matching.loc[brand_matching['score']<0.92, 'brand_abbrev'] = np.nan
brand_matching.loc[brand_matching['score']<0.92, 'elc_brand'] = np.nan

In [37]:
brand_matching.drop('score', axis=1, inplace=True)

In [38]:
# We manually check that we didn't miss anything
pd.set_option('display.max_rows', len(brand_matching)+1)
display(brand_matching)
pd.set_option('display.max_rows', 100)

,brand,brand_abbrev,elc_brand
0,/skin regimen/,NaN,NaN
1,100% pure,NaN,NaN
2,1001 remedies,NaN,NaN
3,111skin,NaN,NaN
4,2(x)ist,NaN,NaN
5,27 rosiers,NaN,NaN
6,29 by lydia mondavi,NaN,NaN
7,3 concept eyes,NaN,NaN
8,37 actives,NaN,NaN
9,37 extreme actives,NaN,NaN


In [39]:
brand_matching.to_csv('../data/clean_data/brand_mapping.csv')

## Aggregate by Brand - ELC solution type - year - month

In [63]:
brand_matching = pd.read_csv('../data/clean_data/brand_mapping.csv')
ratings = pd.read_csv('../data/clean_data/ratings_and_reviews_by_product_id.csv')

Now we want to aggregate the RR data by brand, elc_solution_type and date to link it to the Demand data. We only keep the ont-hot encoded data and aggregate it using sum. But first, we add the ELC brand mapping :

In [64]:
ratings = ratings.merge(brand_matching, how='left', on='brand')

All the missing brands are brands that do not belong to ELC so we drop them.

In [65]:
ratings.isna().sum()

elc_solution_type                  0
source_product_identifier          0
product_id                         0
year                               0
month                              0
rating                           269
rating_1                           0
rating_2                           0
rating_3                           0
rating_4                           0
rating_5                           0
sentiment_negative                 0
sentiment_neutral                  0
sentiment_positive                 0
sentiment                          0
brand                          12988
major_category                     0
Unnamed: 0                     12988
brand_abbrev                 1961304
elc_brand                    1961304
dtype: int64

In [66]:
ratings.dropna(subset=['elc_brand'], inplace=True)

In [67]:
ratings

,elc_solution_type,source_product_identifier,product_id,year,month,rating,rating_1,rating_2,rating_3,rating_4,rating_5,sentiment_negative,sentiment_neutral,sentiment_positive,sentiment,brand,major_category,Unnamed: 0,brand_abbrev,elc_brand
235,After Sun,B0013EKPK2,Product_20191016_13533532,2015,7,5.0,0,0,0,0,2,0,0,2,1.000000,clinique,Skincare,319.0,CL,Clinique
236,After Sun,B0013EKPK2,Product_20191016_13533532,2016,6,5.0,0,0,0,0,2,0,0,2,1.000000,clinique,Skincare,319.0,CL,Clinique
237,After Sun,B0013EKPK2,Product_20191016_13533532,2018,5,5.0,0,0,0,0,5,0,1,4,0.800000,clinique,Skincare,319.0,CL,Clinique
238,After Sun,B0013EKPK2,Product_20191016_13533532,2018,7,5.0,0,0,0,0,3,0,0,3,1.000000,clinique,Skincare,319.0,CL,Clinique
239,After Sun,B0013EKPK2,Product_20191016_13533532,2018,10,5.0,0,0,0,0,3,0,0,3,1.000000,clinique,Skincare,319.0,CL,Clinique
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2251641,Unspecified Lip Makeup,3521576,Product_20200109_5966073,2020,2,5.0,0,0,0,0,4,0,2,2,0.500000,mac,Cosmetics,894.0,MC,M.A.C
2251642,Unspecified Lip Makeup,4428719,Product_20200109_5818167,2020,2,5.0,0,0,0,0,2,0,0,2,1.000000,mac,Cosmetics,894.0,MC,M.A.C
2251665,Unspecified Lip Makeup,B00ITSYQJ6,Product_20200109_5707932,2020,2,5.0,0,0,0,0,6,0,1,5,0.833333,bobbi brown,Cosmetics,230.0,BB,Bobbi Brown
2251680,Unspecified Lip Makeup,B01CQEJP44,Product_20200109_5706849,2020,2,5.0,0,0,0,0,2,0,0,2,1.000000,bobbi brown,Cosmetics,230.0,BB,Bobbi Brown


In [68]:
# This was a quick fix because I didn't want to rerun the code from the beginning to add this column
# ratings['nb_reviews'] = ratings[['rating_1', 'rating_2', 'rating_3', 'rating_4','rating_5']].sum(1)

Finally, we aggregate the data :

In [69]:
ratings = ratings.groupby(['elc_brand',
                           'brand_abbrev',
                           'elc_solution_type', 
                           'year',
                           'month']).agg({
        'nb_reviews':'sum',
        'rating':'mean',
        'sentiment':'mean',
        'rating_1':'sum',
        'rating_2':'sum',
        'rating_3':'sum',
        'rating_4':'sum',
        'rating_5':'sum',
        'sentiment_negative':'sum',
        'sentiment_neutral':'sum',
        'sentiment_positive':'sum',
    }).reset_index().rename(columns={
    'brand_abbrev':'brand',
    'elc_solution_type':'sub_category'
})

In [70]:
ratings

,elc_brand,brand,sub_category,year,month,nb_reviews,rating,sentiment,rating_1,rating_2,rating_3,rating_4,rating_5,sentiment_negative,sentiment_neutral,sentiment_positive
0,Aramis,AR,Cleanser,2015,5,6,5.000000,1.000000,0,0,0,0,6,0,0,6
1,Aramis,AR,Cleanser,2016,2,9,5.000000,1.000000,0,0,0,0,9,0,0,9
2,Aramis,AR,Cleanser,2018,4,12,1.000000,0.500000,12,0,0,0,0,0,6,6
3,Aramis,AR,Cleanser,2020,1,3,5.000000,0.666667,0,0,0,0,3,0,1,2
4,Aramis,AR,Cleansers (Incl Soap),2017,7,7,5.000000,1.000000,0,0,0,0,7,0,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21834,Tom Ford Beauty,TF,Unspecified Lip Skincare,2019,4,50,5.000000,0.800000,0,0,0,0,50,0,10,40
21835,Tom Ford Beauty,TF,Unspecified Lip Skincare,2019,10,27,5.000000,0.666667,0,0,0,0,27,0,9,18
21836,Tom Ford Beauty,TF,Unspecified Lip Skincare,2019,12,21,3.666667,0.682540,6,0,0,0,15,3,0,18
21837,Tom Ford Beauty,TF,Unspecified Lip Skincare,2020,1,45,4.610837,0.685530,0,0,10,0,35,2,12,31


In [71]:
# We format the date in the same way of the demand data
ratings['date'] = pd.to_datetime(ratings[['year', 'month']].assign(DAY=1))
ratings.drop(['year', 'month'], axis=1, inplace=True)
ratings

,elc_brand,brand,sub_category,nb_reviews,rating,sentiment,rating_1,rating_2,rating_3,rating_4,rating_5,sentiment_negative,sentiment_neutral,sentiment_positive,date
0,Aramis,AR,Cleanser,6,5.000000,1.000000,0,0,0,0,6,0,0,6,2015-05-01
1,Aramis,AR,Cleanser,9,5.000000,1.000000,0,0,0,0,9,0,0,9,2016-02-01
2,Aramis,AR,Cleanser,12,1.000000,0.500000,12,0,0,0,0,0,6,6,2018-04-01
3,Aramis,AR,Cleanser,3,5.000000,0.666667,0,0,0,0,3,0,1,2,2020-01-01
4,Aramis,AR,Cleansers (Incl Soap),7,5.000000,1.000000,0,0,0,0,7,0,0,7,2017-07-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21834,Tom Ford Beauty,TF,Unspecified Lip Skincare,50,5.000000,0.800000,0,0,0,0,50,0,10,40,2019-04-01
21835,Tom Ford Beauty,TF,Unspecified Lip Skincare,27,5.000000,0.666667,0,0,0,0,27,0,9,18,2019-10-01
21836,Tom Ford Beauty,TF,Unspecified Lip Skincare,21,3.666667,0.682540,6,0,0,0,15,3,0,18,2019-12-01
21837,Tom Ford Beauty,TF,Unspecified Lip Skincare,45,4.610837,0.685530,0,0,10,0,35,2,12,31,2020-01-01


In [72]:
ratings.to_csv(os.path.join(output_ratings_path,'ratings_and_reviews_by_brand_sub_category.csv'), index=False)

## Formating Demand Data

Before this code, I manually deleted the first row of the xlsx file and saved it as a .csv file. A lot of formating needs to be done to transform the data into a Pandas friendly DataFrame.

In [87]:
# Read the data
demand = pd.read_csv(os.path.join(input_demand_paths,'NA+UK demand data by category FY17-FY20.csv'))
demand.tail(3)

,Brand,ItemID 4,Item Description,Affiliate,Major Category,Category,Sub Category,Application,7/1/2016,8/1/2016,9/1/2016,10/1/2016,11/1/2016,12/1/2016,1/1/2017,2/1/2017,3/1/2017,4/1/2017,5/1/2017,6/1/2017,7/1/2017,8/1/2017,9/1/2017,10/1/2017,11/1/2017,12/1/2017,1/1/2018,2/1/2018,3/1/2018,4/1/2018,5/1/2018,6/1/2018,7/1/2018,8/1/2018,9/1/2018,10/1/2018,11/1/2018,12/1/2018,1/1/2019,2/1/2019,3/1/2019,4/1/2019,5/1/2019,6/1/2019,7/1/2019,8/1/2019,9/1/2019,10/1/2019,11/1/2019,12/1/2019,1/1/2020,2/1/2020,3/1/2020,4/1/2020,5/1/2020,6/1/2020
42423,-,J0TK,-,Canada,Fragrance,All Other Fragrance,All Other Fragrance,All Other Fragrance,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,1,1
42424,-,J0TK,-,UK AFFILIATE,Fragrance,All Other Fragrance,All Other Fragrance,All Other Fragrance,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,8,4,4
42425,-,J0TK,-,US,Fragrance,All Other Fragrance,All Other Fragrance,All Other Fragrance,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,19,51,63,55,-,27,28,63,-,24,21,12,12,-,12,12


In [88]:
# Pandas friendly column names
demand.columns = [col.lower().replace(' ','_') for col in demand.columns]

# We extract the columns that contain demand data. They have the format (dd/mm/yyy)
# TODO: use regex. Bleh
dates_columns = demand.columns[demand.columns.str.contains('/')].tolist()

# In xlsx 0 is marked as '-' which cause the demand columns to be read as of type object (str).
# We need to replace the '-' by 0 and transform the type to int so that we can aggregate using sum().
demand[demand == '-']=0
demand.loc[:, dates_columns] = demand.loc[:, dates_columns].apply(lambda x: x.str.replace(',', '').fillna(0).astype(int), axis=0)

# We aggregate the data
demand = demand.groupby(['brand', 'sub_category'])[dates_columns].sum()

# Finally we use stack to create a columns date, and have a row for each seperate month
demand.columns.name = 'date'
demand = demand.stack().to_frame('demand').reset_index()
demand['date'] = pd.to_datetime(demand['date'], errors='coerce')

demand

,brand,sub_category,date,demand
0,0,,2016-07-01,0
1,0,,2016-08-01,0
2,0,,2016-09-01,0
3,0,,2016-10-01,0
4,0,,2016-11-01,0
...,...,...,...,...
47851,TF,Volumizing,2020-02-01,1573
47852,TF,Volumizing,2020-03-01,1112
47853,TF,Volumizing,2020-04-01,280
47854,TF,Volumizing,2020-05-01,158


In [75]:
demand.to_csv(os.path.join(output_demand_path,'demand.csv'), index=False)

## Merging Rating and Reviews with Demand data

In [92]:
ratings

,elc_brand,brand,sub_category,nb_reviews,rating,sentiment,rating_1,rating_2,rating_3,rating_4,rating_5,sentiment_negative,sentiment_neutral,sentiment_positive,date
0,Aramis,AR,Cleanser,6,5.000000,1.000000,0,0,0,0,6,0,0,6,2015-05-01
1,Aramis,AR,Cleanser,9,5.000000,1.000000,0,0,0,0,9,0,0,9,2016-02-01
2,Aramis,AR,Cleanser,12,1.000000,0.500000,12,0,0,0,0,0,6,6,2018-04-01
3,Aramis,AR,Cleanser,3,5.000000,0.666667,0,0,0,0,3,0,1,2,2020-01-01
4,Aramis,AR,Cleansers (Incl Soap),7,5.000000,1.000000,0,0,0,0,7,0,0,7,2017-07-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21834,Tom Ford Beauty,TF,Unspecified Lip Skincare,50,5.000000,0.800000,0,0,0,0,50,0,10,40,2019-04-01
21835,Tom Ford Beauty,TF,Unspecified Lip Skincare,27,5.000000,0.666667,0,0,0,0,27,0,9,18,2019-10-01
21836,Tom Ford Beauty,TF,Unspecified Lip Skincare,21,3.666667,0.682540,6,0,0,0,15,3,0,18,2019-12-01
21837,Tom Ford Beauty,TF,Unspecified Lip Skincare,45,4.610837,0.685530,0,0,10,0,35,2,12,31,2020-01-01


In [76]:
rating_demand = ratings.merge(demand)

In [98]:
# Ordering columns and rows
rating_demand = rating_demand.set_index(['elc_brand', 'brand', 'sub_category','date']).reset_index()
rating_demand

,elc_brand,brand,sub_category,date,nb_reviews,rating,sentiment,rating_1,rating_2,rating_3,rating_4,rating_5,sentiment_negative,sentiment_neutral,sentiment_positive,demand
0,Aveda,AV,All Exfoliators,2018-02-01,11,1.000000,0.727273,11,0,0,0,0,1,1,9,12752
1,Aveda,AV,All Exfoliators,2018-04-01,17,5.000000,0.823529,0,0,0,0,17,1,1,15,15947
2,Aveda,AV,All Exfoliators,2018-05-01,5,4.000000,1.000000,0,0,0,5,0,0,0,5,28376
3,Aveda,AV,All Exfoliators,2018-07-01,7,4.500000,0.800000,0,0,0,2,5,0,2,5,16646
4,Aveda,AV,All Exfoliators,2018-08-01,14,5.000000,0.714286,0,0,0,0,14,2,0,12,19970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
740,Tom Ford Beauty,TF,All Palettes,2019-08-01,62,4.794872,0.903846,0,0,16,0,46,7,1,54,0
741,Tom Ford Beauty,TF,Makeup Primers,2019-03-01,18,5.000000,0.791667,0,0,0,0,18,1,2,15,0
742,Tom Ford Beauty,TF,Makeup Primers,2019-04-01,45,4.151515,0.851010,0,0,16,9,20,1,5,39,0
743,Tom Ford Beauty,TF,Makeup Primers,2019-05-01,11,3.909091,0.727273,0,4,0,0,7,1,1,9,0


In [78]:
rating_demand.to_csv(os.path.join(output_demand_path,'rating_demand.csv'), index=False)